![lop](../../images/logo_diive1_128px.png)

<span style='font-size:32px; display:block;'>
<b>
    Calculate VPD from air temperature and relative humidity
</b>
</span>

---
**Notebook version**: `2` (6 Sep 2023)  
Example notebook for function `pkgs.createvar.vpd.calc_vpd_from_ta_rh` of the time series processing library `diive`.  
**Author**: Lukas Hörtnagl (holukas@ethz.ch)  

</br>

# **Description**

- Air temperature (TA) and relative humidity (RH) are two widely available measurements
- VPD can be calculated from TA and RH
- In this example, gap-filled TA is used in combination with RH (not gap-filled) to calculate VPD 
- `QuickFillRFTS` is used for quick gap-filling of the calculated VPD. It uses the input features in combination with timestamp info.

</br>

# **Imports**

In [ ]:
import pkg_resources
from datetime import datetime
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from diive.configs.exampledata import load_exampledata_pickle
from diive.core.times.times import insert_timestamp  # Used to insert TIMESTAMP_END
from diive.core.plotting.heatmap_datetime import HeatmapDateTime  # Used for plotting
from diive.pkgs.gapfilling.randomforest_ts import QuickFillRFTS  # Used for gap-filling
from diive.pkgs.createvar.vpd import calc_vpd_from_ta_rh  # Used to calculate VPD

dt_string = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"This page was last modified on: {dt_string}")
version_diive = pkg_resources.get_distribution("diive").version
print(f"diive version: v{version_diive}")

</br>

# **Docstring**

In [ ]:
print(calc_vpd_from_ta_rh.__doc__)

</br>

# **Variables**
- Names of the required variables in the datafiles

In [ ]:
ta_col = 'Tair_f'  # Air temperature (gap-filled) is used to calculate VPD
rh_col = 'RH'  # Relative humidity (not gap-filled) is used to calculate VPD

</br>

# **Newly created variables**

In [ ]:
vpd_col = 'VPD_hPa'  # VPD will be newly calculated from gap-filled TA and non-gap-filled RH
vpd_gf_col = f'{vpd_col}_gfRF'  # Name of the gap-filled VPD in output

</br>

# **Load example data**

In [ ]:
df = load_exampledata_pickle()

In [ ]:
df

</br>

# **Subset of required variables**

In [ ]:
subsetcols = [ta_col, rh_col]
subset_df = df[subsetcols].copy()
subset_df

</br>

# **Calculation of VPD**

In [ ]:
subset_df[vpd_col] = calc_vpd_from_ta_rh(df=subset_df, ta_col=ta_col, rh_col=rh_col)

In [ ]:
subset_df

</br>

# **Gap-filling VPD**
- Quick gap-filling, just using air temperature and timestamp info as predictors

In [ ]:
gfcols = [vpd_col, ta_col]
gf_df = subset_df[gfcols].copy()
rfts = QuickFillRFTS(df=gf_df, target_col=vpd_col)
rfts.fill()
subset_df[vpd_gf_col] = rfts.get_gapfilled()

In [ ]:
subset_df

</br>

# **Plot some variables**

In [1]:
fig = plt.figure(facecolor='white', figsize=(16, 9))
gs = gridspec.GridSpec(1, 3)  # rows, cols
ax_fc = fig.add_subplot(gs[0, 0])
ax_le = fig.add_subplot(gs[0, 1])
ax_ta = fig.add_subplot(gs[0, 2])
HeatmapDateTime(ax=ax_fc, series=subset_df['VPD_hPa_gfRF']).plot()
HeatmapDateTime(ax=ax_le, series=subset_df['VPD_hPa']).plot()
HeatmapDateTime(ax=ax_ta, series=subset_df['Tair_f']).plot()
ax_fc.set_title("Gap-filled VPD", color='black')
ax_le.set_title("VPD", color='black')
ax_ta.set_title("TA", color='black')
ax_le.tick_params(left=True, right=False, top=False, bottom=True,
                     labelleft=False, labelright=False, labeltop=False, labelbottom=True)
ax_ta.tick_params(left=True, right=False, top=False, bottom=True,
                     labelleft=False, labelright=False, labeltop=False, labelbottom=True)

NameError: name 'plt' is not defined

</br>

# **Insert TIMESTAMP_END**
- `diive` uses TIMESTAMP_MIDDLE for calculations, but sometimes TIMESTAMP_END is required for output files

In [ ]:
subset_df = insert_timestamp(data=subset_df, convention='end')
subset_df

</br>

# **Plot**

In [ ]:
subset_df.plot(subplots=True)
plt.show()

</br>

# **Save to CSV file**

In [ ]:
outfile = "CH-FRU_VPD_RH_TA_2020-2022_20230615.csv"
subset_df.to_csv(outfile, index=False)

</br>

# **End of notebook**

In [ ]:
dt_string = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Finished {dt_string}")